In [1]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')
objects = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга11.xlsx')

In [4]:
stock.drop_duplicates(inplace = True)

In [5]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 12/12 [12:18<00:00, 61.52s/it]


In [6]:
objects_with_sales = objects['Код объекта'].unique().tolist()

sale = sale.query('КБК in @objects_with_sales')
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-19', dayfirst=True, format='mixed'))]

In [7]:
compare_tov_id = stock[['Код товара', 'ID Сайта']]
compare_tov_id.columns = ['Тов', 'ID Сайта']

In [8]:
sale = sale.merge(compare_tov_id, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]
sale.dropna(inplace=True)
sale = sale.groupby(by = 'ID Сайта', as_index=False)[['Количество', 'Общая сумма']].sum()
sale.columns = ['ID Сайта', 'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06']

MemoryError: Unable to allocate 9.49 GiB for an array with shape (3, 424396846) and data type object

In [ ]:
stock = stock.merge(sale, how='outer', on='ID Сайта')
result = stock.drop_duplicates(subset=['ID Сайта'])

In [ ]:
ids = result['ID Сайта'].dropna().tolist()

In [ ]:
images['article'] = images['article'].astype(str)
images = images.query('article in @ids')

In [ ]:
images = images[['article', 'published', 'Ссылка на изображение']]
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']
result = result.merge(images, how='outer', on='ID Сайта')

In [ ]:
result = result.drop_duplicates(subset=['ID Сайта'])
result = result[result['ID Сайта'] != '']

In [ ]:
result = result.drop(columns=['Остаток'])

In [ ]:
df = stock
df = df[df['ID Сайта'] != '']

In [ ]:
stock = stock.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
result = result.merge(stock, how='left', on='ID Сайта')

In [ ]:
df['Чистый вес'] = df['Чистый вес'].astype(float)
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.rename(columns={'median': 'Чистый вес медиана', 'max': 'Чистый вес макс', 'min': 'Чистый вес мин'}, inplace=True)
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [ ]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена Розн., за шт'].agg({'min', 'median', 'max'})
price_birka.rename(columns={'median': 'Цена изделия на бирке медиана', 'max': 'Цена изделия на бирке макс', 'min': 'Цена изделия на бирке мин'}, inplace=True)
result = result.merge(price_birka, how='left', on='ID Сайта')
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) 

In [ ]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
price_birka.rename(columns={'max': 'Базовая скидка макс', 'min': 'Базовая скидка мин'}, inplace=True)
result = result.merge(base_discount, how='left', on='ID Сайта')

In [ ]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии'] / 100) * 0.8 * 0.6
)

In [ ]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.rename(columns={'median': 'Цена со скидками медиана', 'max': 'Цена со скидками макс', 'min': 'Цена со скидками мин'}, inplace=True)
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [ ]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.rename(columns={'median': 'Себестоимость медиана', 'max': 'Себестоимость макс', 'min': 'Себестоимость мин'}, inplace=True)
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) 
result = result.merge(selfprice, how='left', on='ID Сайта')

In [ ]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.rename(columns={'median': 'Наценка медиана', 'max': 'Наценка макс', 'min': 'Наценка мин'}, inplace=True)
result = result.merge(markup, how='left', on='ID Сайта')

In [ ]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)
zapret = zapret['Артикул сайта'].tolist()

result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [ ]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.24.xlsx')

In [ ]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [ ]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [ ]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [ ]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'size', 'Цена со скидками Ecom']]

In [ ]:
ecom = ecom.query('article in @ids')

In [ ]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [ ]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [ ]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.rename(columns={'median': 'Цена за грамм медиана', 'max': 'Цена за грамм макс', 'min': 'Цена за грамм мин'}, inplace=True)
result = result.merge(price_per_gramm, how='left', on='ID Сайта')
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) 

In [ ]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [ ]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

yes = yes.merge(no, how='outer', on='ID Сайта')

result = result.merge(yes, how='left', on='ID Сайта')

In [ ]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [ ]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()


data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [ ]:
data = data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'])
result = result.merge(data, how='left', on='ID Сайта')

In [ ]:
result = result.drop(columns=['Дата закупки_x'])

In [ ]:
result.rename(columns={'Дата закупки_y': 'Дата закупки'}, inplace=True)

In [ ]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [ ]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [ ]:
result = result.drop(columns = ['Код склада', 'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки'])

In [ ]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж 14.03-19.06'].sum()

result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'][i] = result['Остаток'][i] / tov[t] 
    
    
result['Доля % относительно продаж ТГ 14.03-19.06'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ 14.03-19.06'][i] = result['Количество продаж 14.03-19.06'][i] / tov_sales[t] 

In [ ]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)